In [1]:
# Engagement Model Template

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools

# 1 -- Load Data

In [3]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [4]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [5]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [6]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [7]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [8]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [9]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [10]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [11]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [12]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

### Filter Feature Set (Optional)

In [13]:
# features_to_keep = data[0].columns

# always include basic_cols, add desired group of features 
features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [14]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [15]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [16]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for training
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i])
    
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i], split_size)
            train_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i])
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)

    y_train = train_data['engagement']
    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])
    y_test = test_data['engagement']
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    # Sort Training Data Together 
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)
    X_train, bogus1, y_train, bogus2 = train_test_split(X_train, y_train, test_size=0.0, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        X_train[c] = (X_train[c]-mean)/(std)
        X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [17]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.naive_bayes import GaussianNB

# Hyperparameter Combinations to Try:  
hp_names = ['var_smoothing']
hp_list = [[1e-12,1e-9,1e-8,1e-6]]

def model(X_train, y_train, X_test, y_test, hp, isVerbose):
    # Model Here: must create pred & scores arrays
    clf = GaussianNB(var_smoothing=hp['var_smoothing'])
    clf.fit(X_train.values, y_train.values)

    pred = clf.predict(X_test.values)
    scores = clf.predict_proba(X_test.values)[:,1]
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if isVerbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc
    
        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [18]:
all_results = []
for n in range(1, len(part)):
    comb = combinations(part, n)
    print()
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(train_part, test_part, 0)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        
        # Set Up Cross Validation Groups
        div = int(len(X_train)/3)
        X_cv1 = X_train.loc[:div]
        X_cv2 = X_train.loc[div+1:div*2]
        X_cv3 = X_train.loc[(div*2)+1:]
        
        y_cv1 = y_train.loc[:div]
        y_cv2 = y_train.loc[div+1:div*2]
        y_cv3 = y_train.loc[(div*2)+1:]
        
        cv1_X_train = X_cv1.append(X_cv2)
        cv1_X_test = X_cv3
        cv1_y_train = y_cv1.append(y_cv2)
        cv1_y_test = y_cv3
        
        cv2_X_train = X_cv1.append(X_cv3)
        cv2_X_test = X_cv2
        cv2_y_train = y_cv1.append(y_cv3)
        cv2_y_test = y_cv2

        cv3_X_train = X_cv3.append(X_cv2)
        cv3_X_test = X_cv1
        cv3_y_train = y_cv3.append(y_cv2)
        cv3_y_test = y_cv1

        # Hyperparameter Combinations        
        hp_combos = list(itertools.product(*hp_list))
        best, best_acc = None, 0
        for h in hp_combos:
            hp = {}
            for i,v in enumerate(h):
                hp[hp_names[i]] = v
            
            # Cross-Validation
            res1 = model(cv1_X_train, cv1_y_train, cv1_X_test, cv1_y_test, hp, False)
            res2 = model(cv2_X_train, cv2_y_train, cv2_X_test, cv2_y_test, hp, False)
            res3 = model(cv3_X_train, cv3_y_train, cv3_X_test, cv3_y_test, hp, False)
            
            acc = np.mean([res1['accuracy'], res2['accuracy'], res3['accuracy']])
            print(hp, acc)
            if acc > best_acc:
                best = hp
                best_acc = acc
        
        # Evaluate on Test Using Best Hyperaparemeters  
        print('Best HP', best)
        results = model(X_train, y_train, X_test, y_test, best, True)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(results)



Train: [5] 	Test: [7, 9, 11, 12, 17, 18]
{'var_smoothing': 1e-12} 0.6600907515257514
{'var_smoothing': 1e-09} 0.6621003054903887
{'var_smoothing': 1e-08} 0.6766340186711908
{'var_smoothing': 1e-06} 0.7167615523136396
Best HP {'var_smoothing': 1e-06}
{'recall_0': 0.6610614565515912, 'precision_0': 0.5253886804013173, 'f1_0': 0.5854678515420805, 'recall_1': 0.6521665918275708, 'accuracy': 0.6554405753150133, 'auc': 0.6852384786064595, 'f1_1': 0.7052022244308052, 'precision_1': 0.7676273845899744}

Train: [7] 	Test: [5, 9, 11, 12, 17, 18]
{'var_smoothing': 1e-12} 0.7366094560127566
{'var_smoothing': 1e-09} 0.7380007406161098
{'var_smoothing': 1e-08} 0.7420751513518259
{'var_smoothing': 1e-06} 0.7568322636055324
Best HP {'var_smoothing': 1e-06}
{'recall_0': 0.45202233880521236, 'precision_0': 0.6194820255121762, 'f1_0': 0.522666492727154, 'recall_1': 0.8596241051821008, 'accuracy': 0.7227505209319947, 'auc': 0.6630749027480857, 'f1_1': 0.8046397394484636, 'precision_1': 0.756266465939029

{'recall_0': 0.5938454066056065, 'precision_0': 0.6072226755117245, 'f1_0': 0.6004595443144547, 'recall_1': 0.8270324314191089, 'accuracy': 0.754631823861433, 'auc': 0.7487439191641991, 'f1_1': 0.8229505903201486, 'precision_1': 0.8189088434111405}

Train: [7, 18] 	Test: [5, 9, 11, 12, 17]
{'var_smoothing': 1e-12} 0.7428219929210448
{'var_smoothing': 1e-09} 0.7428219929210448
{'var_smoothing': 1e-08} 0.7427963104082677
{'var_smoothing': 1e-06} 0.7426422331392027
Best HP {'var_smoothing': 1e-12}
{'recall_0': 0.5895193698762257, 'precision_0': 0.6560409287682015, 'f1_0': 0.6210037930100244, 'recall_1': 0.827282036541329, 'accuracy': 0.7420477595426885, 'auc': 0.7354469467142012, 'f1_1': 0.8044898672257161, 'precision_1': 0.7829199041093562}

Train: [9, 11] 	Test: [5, 7, 12, 17, 18]
{'var_smoothing': 1e-12} 0.7642235631506606
{'var_smoothing': 1e-09} 0.7633221941525085
{'var_smoothing': 1e-08} 0.7629044948076568
{'var_smoothing': 1e-06} 0.7622669359983528
Best HP {'var_smoothing': 1e-12}


{'var_smoothing': 1e-12} 0.7735231993334873
{'var_smoothing': 1e-09} 0.7733520326960273
{'var_smoothing': 1e-08} 0.7731979776240917
{'var_smoothing': 1e-06} 0.7731979776240917
Best HP {'var_smoothing': 1e-12}
{'recall_0': 0.5287076486102658, 'precision_0': 0.5971859388606261, 'f1_0': 0.5608643250688705, 'recall_1': 0.7938789868667917, 'accuracy': 0.696752619454559, 'auc': 0.6885608121250462, 'f1_1': 0.7684153179125145, 'precision_1': 0.7445343773369111}

Train: [5, 9, 12] 	Test: [7, 11, 17, 18]
{'var_smoothing': 1e-12} 0.7754082055478076
{'var_smoothing': 1e-09} 0.775290598844169
{'var_smoothing': 1e-08} 0.7753886092332701
{'var_smoothing': 1e-06} 0.7752710002243185
Best HP {'var_smoothing': 1e-12}
{'recall_0': 0.4458239738615479, 'precision_0': 0.5579350881676463, 'f1_0': 0.4956186152099887, 'recall_1': 0.8277228906063262, 'accuracy': 0.702526476455703, 'auc': 0.6553468729348604, 'f1_1': 0.7890589396931489, 'precision_1': 0.7538458747550976}

Train: [5, 9, 17] 	Test: [7, 11, 12, 18]
{

{'recall_0': 0.7033779576587795, 'precision_0': 0.7010045378737928, 'f1_0': 0.7021892422152722, 'recall_1': 0.7974779981610404, 'accuracy': 0.7595552140022898, 'auc': 0.7992429749597555, 'f1_1': 0.7983903843877806, 'precision_1': 0.7993048607088314}

Train: [7, 12, 17] 	Test: [5, 9, 11, 18]
{'var_smoothing': 1e-12} 0.6966352114629091
{'var_smoothing': 1e-09} 0.7006376059232844
{'var_smoothing': 1e-08} 0.7034021429912692
{'var_smoothing': 1e-06} 0.702597508809109
Best HP {'var_smoothing': 1e-08}
{'recall_0': 0.5821973769081917, 'precision_0': 0.5870441833239388, 'f1_0': 0.5846107344876723, 'recall_1': 0.8235577457482678, 'accuracy': 0.750883689614543, 'auc': 0.7324999237806329, 'f1_1': 0.8220956467063654, 'precision_1': 0.8206387299243123}

Train: [7, 12, 18] 	Test: [5, 9, 11, 17]
{'var_smoothing': 1e-12} 0.7393885511793391
{'var_smoothing': 1e-09} 0.7393885511793391
{'var_smoothing': 1e-08} 0.7394618918467392
{'var_smoothing': 1e-06} 0.7403969772882536
Best HP {'var_smoothing': 1e-06}


{'var_smoothing': 1e-12} 0.738124125540328
{'var_smoothing': 1e-09} 0.737586122339209
{'var_smoothing': 1e-08} 0.737344016014652
{'var_smoothing': 1e-06} 0.7380299397061555
Best HP {'var_smoothing': 1e-12}
{'recall_0': 0.4206042707860064, 'precision_0': 0.6765941896583226, 'f1_0': 0.5187364292218253, 'recall_1': 0.89509867836188, 'accuracy': 0.7324064337734159, 'auc': 0.6756114092294325, 'f1_1': 0.8146829570893006, 'precision_1': 0.7475252425262324}

Train: [5, 7, 11, 12] 	Test: [9, 17, 18]
{'var_smoothing': 1e-12} 0.7538266732369827
{'var_smoothing': 1e-09} 0.7538266732369827
{'var_smoothing': 1e-08} 0.7538127203975798
{'var_smoothing': 1e-06} 0.7543010721129572
Best HP {'var_smoothing': 1e-06}
{'recall_0': 0.6245876794151733, 'precision_0': 0.6542160799327669, 'f1_0': 0.6390586518288789, 'recall_1': 0.7656551593203177, 'accuracy': 0.7070895867646242, 'auc': 0.7672090872946112, 'f1_1': 0.7535423998006914, 'precision_1': 0.7418069223458721}

Train: [5, 7, 11, 17] 	Test: [9, 12, 18]
{'v

{'var_smoothing': 1e-12} 0.741568869942653
{'var_smoothing': 1e-09} 0.7418274396826058
{'var_smoothing': 1e-08} 0.7421598860363147
{'var_smoothing': 1e-06} 0.7426893338129635
Best HP {'var_smoothing': 1e-06}
{'recall_0': 0.40887796636203055, 'precision_0': 0.5186556259132976, 'f1_0': 0.45727046293910506, 'recall_1': 0.8429833481632134, 'accuracy': 0.7159297803951539, 'auc': 0.6715621289062372, 'f1_1': 0.8076173658893016, 'precision_1': 0.7750993454885461}

Train: [7, 9, 11, 17] 	Test: [5, 12, 18]
{'var_smoothing': 1e-12} 0.7268562652762894
{'var_smoothing': 1e-09} 0.7240231329411713
{'var_smoothing': 1e-08} 0.7191990770792048
{'var_smoothing': 1e-06} 0.7180249754524981
Best HP {'var_smoothing': 1e-12}
{'recall_0': 0.48954849498327757, 'precision_0': 0.5285727182450122, 'f1_0': 0.5083127143291226, 'recall_1': 0.852435853961795, 'accuracy': 0.7607713102982174, 'auc': 0.6958090350904054, 'f1_1': 0.8419319276015573, 'precision_1': 0.8316837142620827}

Train: [7, 9, 11, 18] 	Test: [5, 12, 1

{'recall_0': 0.5230893325267191, 'precision_0': 0.5430186309399204, 'f1_0': 0.5328677074774034, 'recall_1': 0.8217740947871168, 'accuracy': 0.7356966439052739, 'auc': 0.6985214279811521, 'f1_1': 0.8157137647392521, 'precision_1': 0.8097421664454366}

Train: [5, 7, 9, 11, 18] 	Test: [12, 17]
{'var_smoothing': 1e-12} 0.7426638156125799
{'var_smoothing': 1e-09} 0.7414928554252261
{'var_smoothing': 1e-08} 0.7412052505356704
{'var_smoothing': 1e-06} 0.7418317935307868
Best HP {'var_smoothing': 1e-12}
{'recall_0': 0.5051233707680958, 'precision_0': 0.7294034090909091, 'f1_0': 0.5968905894318788, 'recall_1': 0.8584344810502849, 'accuracy': 0.7063886831057961, 'auc': 0.7342900065532443, 'f1_1': 0.7691069991954949, 'precision_1': 0.6966179204985176}

Train: [5, 7, 9, 12, 17] 	Test: [11, 18]
{'var_smoothing': 1e-12} 0.7316866217955532
{'var_smoothing': 1e-09} 0.7295647600698478
{'var_smoothing': 1e-08} 0.7293621126105679
{'var_smoothing': 1e-06} 0.73018465109824
Best HP {'var_smoothing': 1e-12}


{'var_smoothing': 1e-12} 0.7216538979819599
{'var_smoothing': 1e-09} 0.7171423186329666
{'var_smoothing': 1e-08} 0.7158723939783632
{'var_smoothing': 1e-06} 0.7163736888045965
Best HP {'var_smoothing': 1e-12}
{'recall_0': 0.31593074755066436, 'precision_0': 0.45699864104057464, 'f1_0': 0.37359149341374387, 'recall_1': 0.9018217557653831, 'accuracy': 0.7803561491374513, 'auc': 0.6104507444118545, 'f1_1': 0.8668308647390264, 'precision_1': 0.8344538633927702}

Train: [7, 11, 12, 17, 18] 	Test: [5, 9]
{'var_smoothing': 1e-12} 0.7251915968376079
{'var_smoothing': 1e-09} 0.7251915968376079
{'var_smoothing': 1e-08} 0.7251805215366366
{'var_smoothing': 1e-06} 0.7257896612501851
Best HP {'var_smoothing': 1e-06}
{'recall_0': 0.7652811735941321, 'precision_0': 0.6731182795698925, 'f1_0': 0.7162471395881006, 'recall_1': 0.7711365606606744, 'accuracy': 0.7689049528435082, 'auc': 0.8210864701897272, 'f1_1': 0.8050778831038917, 'precision_1': 0.8421446011261149}

Train: [9, 11, 12, 17, 18] 	Test: [5

In [19]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [20]:
results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.655441,0.685238,0.585468,0.705202,0.525389,0.767627,0.661061,0.652167,"7, 9, 11, 12, 17, 18",5
1,0.722751,0.663075,0.522666,0.804640,0.619482,0.756266,0.452022,0.859624,"5, 9, 11, 12, 17, 18",7
2,0.734444,0.666806,0.434454,0.826484,0.643032,0.751678,0.328047,0.917825,"5, 7, 11, 12, 17, 18",9
3,0.716628,0.765224,0.643890,0.764692,0.603717,0.799861,0.689791,0.732484,"5, 7, 9, 12, 17, 18",11
4,0.721207,0.713295,0.580679,0.791187,0.603430,0.776606,0.559581,0.806325,"5, 7, 9, 11, 17, 18",12


In [21]:
results.to_csv('results.csv', index=False)